# Imports
All problem-speciffic files (code) in in the folder `GinzburgLandau2D`, namely the loading of pre-prepared mesh and definition of energy in jax.numpy.

In [1]:
from tools.minimizers import newton
from tools.sparse_solvers import HessSolverGenerator
from tools.jax_diff import EnergyDerivator as EnergyDerivator
from GinzburgLandau2D.jax_energy import J
from GinzburgLandau2D.mesh import MeshGL2D
from IPython.display import display, HTML
import pandas as pd

from jax import config

config.update("jax_enable_x64", True)

# Benchmark of Ginzburg-Landau 2D

In [2]:
results_df = pd.DataFrame(columns=['Time', 'Nit', 'Fun'])
times_asembly = []
times_solver = []

for mesh_level in range(2, 10):

    mesh = MeshGL2D(mesh_level=mesh_level)
    params, adjacency, u_init = mesh.get_data_jax()
    energy_GL = EnergyDerivator(J, params, adjacency, u_init)

    F, dF, ddF = energy_GL.get_derivatives()
    ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="amg" if mesh_level > 5 else "direct", verbose=False, tol=1e-3)
    res = newton(F, dF, ddf_with_solver, u_init, verbose=False, tolf=1e-6, tolg=1e-5, linesearch_tol=1e-3)
    # Create a DataFrame for the current result, round the results, and concatenate it with the main DataFrame
    time_asembly = energy_GL.timings['compilation'] + energy_GL.timings['coloring']
    times_asembly.append(time_asembly)
    times_solver.append(res['time'])

    current_result_df = pd.DataFrame([{
        'Time': f"{res['time']:.2f} / {time_asembly:.2f}",
        'Nit': res['nit'],
        'Fun': round(float(res['fun']), 4)
    }])
    results_df = pd.concat([results_df, current_result_df], ignore_index=True)

# Display the DataFrame as HTML in Jupyter Notebook
results_df.index = range(2,10)  # Reset index to simple counter
display(HTML(results_df.to_html()))

# Apply rounding for LaTeX export using Styler.format
latex_style = results_df.style.format({
    'Fun': "{:.4f}"
})

print(latex_style.to_latex())

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


,Time,Nit,Fun
2,0.01 / 0.18,6,0.3867
3,0.02 / 0.18,8,0.3547
4,0.03 / 0.24,7,0.3480
5,0.10 / 0.20,7,0.3462
6,0.40 / 0.26,6,0.3458
7,1.47 / 0.51,6,0.3457
8,6.04 / 1.56,6,0.3456
9,26.63 / 5.76,6,0.3456


\begin{tabular}{lllr}
 & Time & Nit & Fun \\
2 & 0.01 / 0.18 & 6 & 0.3867 \\
3 & 0.02 / 0.18 & 8 & 0.3547 \\
4 & 0.03 / 0.24 & 7 & 0.3480 \\
5 & 0.10 / 0.20 & 7 & 0.3462 \\
6 & 0.40 / 0.26 & 6 & 0.3458 \\
7 & 1.47 / 0.51 & 6 & 0.3457 \\
8 & 6.04 / 1.56 & 6 & 0.3456 \\
9 & 26.63 / 5.76 & 6 & 0.3456 \\
\end{tabular}

